In [ ]:
# Import all necessary libraries
import pandas as pd
import numpy as np
import psycopg2 # needed to get database exception errors when uploading dataframe
import requests # package for getting data from the web
from zipfile import * # package for unzipping zip files
from sql_functions import get_engine

In [ ]:
path ='data/' 

In [ ]:
def download_data(year, month):
    # Get the file from the website https://transtats.bts.gov
    zip_file = f'On_Time_Reporting_Carrier_On_Time_Performance_1987_present_{year}_{month}.zip'
    url = (f'https://transtats.bts.gov/PREZIP/{zip_file}')
    # Download the database
    r = requests.get(f'{url}', verify=False)
    # Save database to local file storage
    with open(path+zip_file, 'wb') as f:
        f.write(r.content)
        print(f'--> zip_file with name: {zip_file} downloaded succesfully.' )

In [ ]:
def extract_zip(year, month):
    # Get the file from the website https://transtats.bts.gov
    zip_file = f'On_Time_Reporting_Carrier_On_Time_Performance_1987_present_{year}_{month}.zip'
    with ZipFile(path+zip_file, 'r') as zip_ref:
        zip_ref.extractall(path)
        csv_file =  zip_ref.namelist()[0]
        print(f'--> zip_file was succesfully extracted to: {csv_file}.' )

In [ ]:
years_list = [2011, 2012] # list of years you want to look at (can of course also be a single year)
months_list = [10, 11] # list of months you want to look at (can of course also be a single month)

# download flights data as zipfile(s)
# we use a nested loop to specify the years and months to define the range of the data we would like to have 
for year in years_list:
    for month in months_list:
        download_data(year, month)
        extract_zip(year, month)

In [ ]:
csv_file_10_12 = 'On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2012_10.csv'

# Read in your data
df_oct_2012 = pd.read_csv(path+csv_file_10_12, low_memory = False)
display(df_oct_2012.shape)
display(df_oct_2012.head())

csv_file_11_12 = 'On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2012_11.csv'

# Read in your data
df_nov_2012 = pd.read_csv(path+csv_file_11_12, low_memory = False)
display(df_nov_2012.shape)
display(df_nov_2012.head())

In [ ]:
# Columns from downloaded file that are to be kept

columns_to_keep = [
                'FlightDate',
                'DepTime',
                'CRSDepTime',
                'DepDelay',
                'ArrTime',
                'CRSArrTime',
                'ArrDelay',
                'Reporting_Airline',
                'Tail_Number',
                'Flight_Number_Reporting_Airline',
                'Origin',
                'Dest',
                'AirTime',
                'ActualElapsedTime',
                'Distance',
                'Cancelled',
                'Diverted'
]

In [ ]:
schema = 'cgn_analytics_24_3' # UPDATE 'TABLE_SCHEMA' based on schema used in class 
engine = get_engine() # assign engine to be able to query against the database

table_name_sql = f'''SELECT COLUMN_NAME 
                    FROM INFORMATION_SCHEMA.COLUMNS 
                    WHERE TABLE_NAME = 'flights'
                    AND TABLE_SCHEMA ='{schema}'
                    ORDER BY ordinal_position'''
c_names = engine.execute(table_name_sql).fetchall()
c_names


In [ ]:
new_column_names=[]
for name in c_names:
    new_column_names.append(name[0])
new_column_names     

In [ ]:
def clean_airline_df(df):
    '''
    Transforms a df made from BTS csv file into a df that is ready to be uploaded to SQL
    Set rows=0 for no filtering
    '''

    # Build dataframe including only the columns you want to keep
    df_airline = df.loc[:,columns_to_keep]
     
    # Clean data types and NULLs
    df_airline['FlightDate']= pd.to_datetime(df_airline['FlightDate'], yearfirst=True)
    df_airline['CRSArrTime']= pd.to_numeric(df_airline['CRSArrTime'], downcast='integer', errors='coerce')
    df_airline['Cancelled']= pd.to_numeric(df_airline['Cancelled'], downcast='integer')
    df_airline['Diverted']= pd.to_numeric(df_airline['Diverted'], downcast='integer')
    df_airline['ActualElapsedTime']= pd.to_numeric(df_airline['ActualElapsedTime'], downcast='integer', errors='coerce')
    
    # Rename columns
    df_airline.columns = new_column_names
    
    return df_airline

In [ ]:
df_oct_2012_clean = clean_airline_df(df_oct_2012)
df_oct_2012_clean.head()

In [ ]:
df_nov_2012_clean = clean_airline_df(df_nov_2012)
df_nov_2012_clean.head()

In [ ]:
def select_airport(df, airports):
    ''' Helper function for filtering the airline dataframe for a subset of airports'''
    df_out = df.loc[(df.origin.isin(airports)) | (df.dest.isin(airports))]
    return df_out


airports=['JFK', 'LGA', 'EWR', 'PHL', 'BOS', 'DCA', 'IAD', 'BWI']

In [ ]:
if len(airports) > 0:
    df_oct_2012_selected_airports = select_airport(df_oct_2012_clean, airports)
else:
    df_selected_airports = df_oct_2012_clean
df_oct_2012_selected_airports.info()

In [ ]:
if len(airports) > 0:
    df_nov_2012_selected_airports = select_airport(df_nov_2012_clean, airports)
else:
    df_selected_airports = df_nov_2012_clean
df_nov_2012_selected_airports.info()

In [ ]:
combined_df = pd.concat([df_oct_2012_selected_airports, df_nov_2012_selected_airports])
#combined_df['flight_date'] = pd.to_datetime(combined_df['date'])
#start_date = '2012-10-22'
#end_date = '2012-11-03'
#filtered_df = combined_df[(combined_df['flight_date'] >= start_date) & (combined_df['flight_date'] <= end_date)]

combined_df.reset_index(drop=True, inplace=True )
#filtered_df.tail(15)
combined_df.tail(10)

In [ ]:
from sql_functions import get_engine

In [ ]:
table_name = 'flights_oct_nov_2012_sandy'
engine = get_engine()
schema = 'cgn_analytics_24_3'

# If the specified table doesn't exist yet, it will be created
# With 'replace', your data will be replaced if the table already exists.
# This may take some time ...

# Write records stored in a dataframe to SQL database
if engine!=None:
    try:
        combined_df.to_sql(name=table_name, # Name of SQL table
                        con=engine, # Engine or connection
                        if_exists='replace', # Drop the table before inserting new values 
                        schema=schema, # Use schmea that was defined earlier
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None

In [ ]:
df = get_dataframe('''select * 
                   from cgn_analytics_24_3.flights_oct_nov_2012_sandy''')

df.head()